# The Final Trajectory

In order to compute the dynamics in SEOBNRv5HM, we will need to implement the corresponding equations of motion and a numerical integration routine.

The equations of motion require as inputs:
- The masses $m_{1,2}$ in normalized masses, i.e $m_1 + m_2 = 1$.
- The spins $\chi_1$, and $\chi_2$ of the binary in dimensionless units.
- The values of the primitive variables $(r,\phi,p_{r_*},p_{\phi})$.
It then computes:
- The partial derivatives of the Hamiltonian w.r.t the positions, $\tfrac{\partial H}{\partial r}$ and momenta $\tfrac{\partial H}{\partial p_{(r_*,\phi)}}$.
- The tortoise parameter $\xi$.
- The Hamiltonian $H$, the circular frequency $\Omega_{\rm circ}$ that are then used to solve for the flux $F_{\phi}$
And returns the time derivative of each of the primitive variables given by Hamilton's equations of motion as well as the prescription for the radiation-reaction.

The numerical integration routine is split into two parts
- An initial sparsely sampled integration that stops at given stopping conditions
- A finely-sampled interpolation of the initial integration from some stepback time to the stopping conditions to calculate the Non-Quasi-Circular (NQC) Coefficients

In this notebook, we will generate three files:
- The set of expressions needed to generate the equations of motions
- The numerical routines needed to generate the trajectory

Finally, the numerical integration requires as inputs:
- The initial conditions, documented in the initial conditions routine
- The stopping conditions, documented in this notebook
- All other inputs needed for the equations of motion 

In [1]:
import sys,os#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "Dynamics"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

# Step : The Equations of Motions

The equations of motion are given by:

\begin{equation*}
\dot{r} = \frac{1}{\nu}\xi \frac{\partial H}{\partial p_{r_{*}}}\\
\dot{\phi} = \Omega\\
\dot{p}_{r_*} = \frac{1}{\nu}\left(-\xi \frac{\partial H}{\partial r} + \frac{ p_{ r_{*} }F_{\phi} }{ p_{\phi} }\right)\\
\dot{p}_{\phi} = \frac{1}{\nu}F_{\phi}
\end{equation*}

Where, $\nu$ is the symmetric mass ratio, $F_{\phi}$ is the gravitational wave flux, $\frac{\partial H}{\partial \zeta}$ is the partial derivative of the Hamiltonian, $H$, with respect to the primitive variable $\zeta = \{r,\phi,p_{r_{*}},p_\phi\}$.  

In [2]:
%%writefile $Ccodesdir/v5HM_Equations_Of_Motion.txt
rdot = (xi/eta)*dHdprstar(m1,m2,r,prstar,pphi,chi1,chi2)
phidot = Omega
prstardot = (1/eta)*(-xi*dHdr(m1,m2,r,prstar,pphi,chi1,chi2) + prstar*F_phi/pphi)
pphidot = F_phi/eta

Overwriting Dynamics/v5HM_Equations_Of_Motion.txt


# Step : $F_{\phi}$

The gravitational wave flux is given as a function of the primitive variables $\{r,\phi,p_{r_{*}},p_{\phi}\}$, the masses $\{m_1,m_2\}$, the spins $\{\chi_{1},\chi_{2}\}$, the rotational frequency $\Omega$, the circular frequency $\Omega_{\rm circ}$, and the Hamiltonian $H$.

$$
F_{\phi} \equiv F_{\phi}(m_1,m_2,r,p_{r_{*}},p_{\phi},\chi_1,\chi_2,\Omega,\Omega_{\rm circ},H)
$$

In [3]:
%%writefile -a $Ccodesdir/v5HM_Equations_Of_Motion.txt
F_phi = flux(m1, m2, r, phi, prstar, pphi, chi1, chi2,Omega,Omega_circ,H)/Omega

Appending to Dynamics/v5HM_Equations_Of_Motion.txt


# Step : $\Omega$

The rotational frequency is given by:

$$
\Omega = \frac{1}{\nu}\frac{\partial H}{\partial p_{\phi}}
$$

In [4]:
%%writefile -a $Ccodesdir/v5HM_Equations_Of_Motion.txt
Omega = (1/eta)*dHdpphi(m1,m2,r,prstar,pphi,chi1,chi2)

Appending to Dynamics/v5HM_Equations_Of_Motion.txt


# Step : $\Omega_{\rm circ}$

The circular frequency is given by:

$$
\Omega = \frac{1}{\nu}\left.\frac{\partial H}{\partial p_{\phi}}\right|_{p_{r_{*}} = 0}
$$

In [5]:
%%writefile -a $Ccodesdir/v5HM_Equations_Of_Motion.txt
Omega_circ = (1/eta)*dHdpphi_preq0(m1,m2,r,pphi,chi1,chi2)

Appending to Dynamics/v5HM_Equations_Of_Motion.txt


# Step : $H$, $\xi$

The Hamiltonian and the tortoise parameter are given as the result of the Hamiltonian computation, which in turn is a function of the primitive variables and the input parameters

In [6]:
%%writefile -a $Ccodesdir/v5HM_Equations_Of_Motion.txt
H , xi = Hamiltonian(m1,m2,r,prstar,pphi,chi1,chi2)

Appending to Dynamics/v5HM_Equations_Of_Motion.txt


# Step : $\eta$

The symmetric mass ratio is defined as follows

$$
\eta = \frac{m_{1}m_{2}}{(m_1 + m_2)^2}
$$

In [7]:
%%writefile -a $Ccodesdir/v5HM_Equations_Of_Motion.txt
eta = m1*m2/(m1 + m2)/(m1 + m2)

Appending to Dynamics/v5HM_Equations_Of_Motion.txt


# Step : Write into python function

In [8]:
with open(os.path.join(Ccodesdir,"v5HM_Equations_Of_Motion.py"), "w") as output:
    output.write("import numpy as np\n")
    output.write("from Radiation.v5HM_Flux_unoptimized import v5HM_unoptimized_flux as flux\n")
    output.write("from Derivatives.v5HM_Hamiltonian_Derivatives_unoptimized import v5HM_unoptimized_omega_circ as dHdpphi_preq0\n")
    output.write("from Derivatives.v5HM_Hamiltonian_Derivatives_unoptimized import v5HM_unoptimized_dH_dpphi as dHdpphi\n")
    output.write("from Derivatives.v5HM_Hamiltonian_Derivatives_unoptimized import v5HM_unoptimized_dH_dprstar as dHdprstar\n")
    output.write("from Derivatives.v5HM_Hamiltonian_Derivatives_unoptimized import v5HM_unoptimized_dH_dr as dHdr\n")
    output.write("from Hamiltonian.v5HM_Hamiltonian_unoptimized import v5HM_unoptimized_hamiltonian as Hamiltonian\n")
    output.write("def v5HM_unoptimized_rhs(t,y,m1,m2,chi1,chi2,verbose = False):\n    r , phi , prstar , pphi = y[0] , y[1] , y[2] , y[3]\n")
    for line in reversed(list(open(os.path.join(Ccodesdir,"v5HM_Equations_Of_Motion.txt"),"r"))):
        output.write("    %s\n" % line.rstrip())
    output.write("    if not verbose:\n        return np.array([rdot,phidot,prstardot,pphidot])\n")
    output.write("    else:\n        return np.array([rdot,phidot,prstardot,pphidot]), F_phi, Omega, Omega_circ, xi, dHdr(m1,m2,r,prstar,pphi,chi1,chi2)/eta, eta, prstar,pphi\n")

import numpy as np
from Dynamics.v5HM_Equations_Of_Motion import v5HM_unoptimized_rhs as rhs_bob
from Dynamics.pyseobnr_equations_of_motion import get_rhs as rhs_true

N = 100000
gt_pert_total = [[],[],[],[]]
gt_pert_O1 = [[],[],[],[]]
gt_pert_O2 = [[],[],[],[]]
gt_pert_gtO3 = [[],[],[],[]]
rng = np.random.default_rng(seed = 50)
eta = rng.random(N)*.25
chi1 = 2.*rng.random(N)-1
chi2 = 2.*rng.random(N)-1
m2 = (1 - np.sqrt(1 - 4*eta))*.5
m1 = (1 + np.sqrt(1 - 4*eta))*.5
r = rng.random(N)*17. + 3.
phi = rng.random(N)*2.*np.pi
prstar = rng.random(N)*20. - 10.
pphi = rng.random(N)*20. - 10.

pert_exponent = 1e-14
pert_sign = 2*(rng.integers(0,high = 1,size = N)) - 1
pert_mantissa = 3.*rng.random(N) + 1.
pert = 1. + pert_sign*pert_mantissa*pert_exponent
chi1pert = chi1*pert 
chi2pert = chi2*pert
m2pert = m2*pert
m1pert = m1*pert
rpert = r*pert
phipert = phi*pert
prstarpert = prstar*pert
pphipert = pphi*pert


disagrmt_max = [0,0,0,0]
disagrmt_max_index = [0,0,0,0]
nans = []
def E_rel(a,b):
    return np.abs((a - b)/a)

for i in range(N):
    dyn_bob = rhs_bob(0.,[r[i],phi[i],prstar[i],pphi[i]],m1[i],m2[i],chi1[i],chi2[i])
    dyn_true = rhs_true(0.,[r[i],phi[i],prstar[i],pphi[i]],chi1[i],chi2[i],m1[i],m2[i])
    dyn_pert = rhs_true(0.,[rpert[i],phipert[i],prstarpert[i],pphipert[i]],chi1pert[i],chi2pert[i],m1pert[i],m2pert[i])
    e_rels = [E_rel(dyn_bob[0],dyn_true[0]),E_rel(dyn_bob[1],dyn_true[1]),E_rel(dyn_bob[2],dyn_true[2]),E_rel(dyn_bob[3],dyn_true[3])]
    tols = [E_rel(dyn_pert[0],dyn_true[0]),E_rel(dyn_pert[1],dyn_true[1]),E_rel(dyn_pert[2],dyn_true[2]),E_rel(dyn_pert[3],dyn_true[3])]
    for j in range(4):
        e_rel , tol = e_rels[j] , tols[j]
        if (e_rel>tol and tol > 0):
            gt_pert_total[j].append(i)
            if e_rel/tol > disagrmt_max[j]:
                disagrmt_max[j] = e_rel/tol
                disagrmt_max_index[j] = i
            if e_rel > 1000*tol:
                gt_pert_gtO3[j].append(i)
            elif e_rel > 100*tol:
                gt_pert_O2[j].append(i)
            elif e_rel > 10*tol:
                gt_pert_O1[j].append(i)
dvar = ['r','phi','prstar','pphi']
for j in range(4):
    print("Of total ",str(N)," comparisons, for %s\n"%dvar[j],
          "number of cases with relative error (total)  greater than allowed: ",len(gt_pert_total[j]),"\n",
          "number of cases with relative error O(10)    greater than allowed: ",len(gt_pert_O1[j]),"\n",
          "number of cases with relative error O(100)   greater than allowed: ",len(gt_pert_O2[j]),"\n",
          "number of cases with relative error O(1000+) greater than allowed: ",len(gt_pert_gtO3[j]))
#results = np.array([eta,r,phi,prstar,pphi,chi1,chi2])
#analytics = np.array([[len(gt_pert_total[0]),len(gt_pert_total[1]),len(gt_pert_total[2]),len(gt_pert_total[2])],[len(gt_pert_O1[0]),len(gt_pert_O1[1]),len(gt_pert_O1[2])],[len(gt_pert_O2[0]),len(gt_pert_O2[1]),len(gt_pert_O2[2])],[len(gt_pert_gtO3[0]),len(gt_pert_gtO3[1]),len(gt_pert_gtO3[2])],disagrmt_max,disagrmt_max_index])
#np.savetxt(os.path.join(outputdir,"eom_validation_results.dat"),results)
#np.savetxt(os.path.join(outputdir,"eom_validation_analytics.dat"),results)
exit(1)

C:\Users\sidmahesh\Documents\GitHub\bob\V5\Radiation\v5HM_Flux_unoptimized.py:16: RuntimeWarning: invalid value encountered in scalar power
  vomega = Omega**(np.divide(1,3))
C:\Users\sidmahesh\Documents\GitHub\bob\V5\Radiation\v5HM_Flux_unoptimized.py:17: RuntimeWarning: invalid value encountered in scalar power
  vphi = Omega*(Omega_circ**(-np.divide(2,3)))
C:\Users\sidmahesh\Documents\GitHub\bob\V5\Radiation\v5HM_Flux_unoptimized.py:348: RuntimeWarning: invalid value encountered in log
  T55 = gamma(5 + 1 - 2*1j*khat5)*np.exp(np.pi*khat5)*(np.exp(2*1j*khat5*np.log(2*5*Omega*r0)))/factorial(5)
C:\Users\sidmahesh\Documents\GitHub\bob\V5\Radiation\v5HM_Flux_unoptimized.py:352: RuntimeWarning: invalid value encountered in log
  T43 = gamma(4 + 1 - 2*1j*khat3)*np.exp(np.pi*khat3)*(np.exp(2*1j*khat3*np.log(2*3*Omega*r0)))/factorial(4)
C:\Users\sidmahesh\Documents\GitHub\bob\V5\Radiation\v5HM_Flux_unoptimized.py:354: RuntimeWarning: invalid value encountered in log
  T44 = gamma(4 + 1 - 2*

Of total  100000  comparisons, for r
 number of cases with relative error (total)  greater than allowed:  1 
 number of cases with relative error O(10)    greater than allowed:  0 
 number of cases with relative error O(100)   greater than allowed:  0 
 number of cases with relative error O(1000+) greater than allowed:  0
Of total  100000  comparisons, for phi
 number of cases with relative error (total)  greater than allowed:  418 
 number of cases with relative error O(10)    greater than allowed:  7 
 number of cases with relative error O(100)   greater than allowed:  0 
 number of cases with relative error O(1000+) greater than allowed:  0
Of total  100000  comparisons, for prstar
 number of cases with relative error (total)  greater than allowed:  10190 
 number of cases with relative error O(10)    greater than allowed:  1564 
 number of cases with relative error O(100)   greater than allowed:  127 
 number of cases with relative error O(1000+) greater than allowed:  13
Of total 

# Step : The Stopping Conditions

The stopping conditions are given in lines 152-178 of [integrate_ode.py](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/eob/dynamics/integrate_ode.py) of pyseobnr.

There are several termination conditions that are checked when $r < 6$:

- Peak orbital frequency : If the value of $\Omega$ has decreased since the last timestep.
- Positive radial velocity: If $\dot{r} > 0$
- Positive radial momentum derivative: If $\dot{p_{r_{*}}} > 0$
- Radius reaches threshold: If the radius reaches some threshold radius (conditionally defined by numerical relativity and the Kerr metric last stable circular orbit)
- Unphysical circular frequency at low $r$'s: if $r < 3M$ and $\Omega_{\rm circ} > 1$

We will define each of these conditions below.

# Step : The First Integration

In the first integration, the initial conditions are evaluated and the trajectory is evolved until the above specified stopping conditions are accomplished. 
pyseobnr uses `pygsl_lite` to access gsl's `odeiv2` tools. We use the equivalent `scipy.odeint` tools.

We use an adaptive Runge-Kutta 45 method with relative tolerance `1e-11` and absolute tolerance `1e-12`

In [9]:
validation = False
if validation:
    import time
    from Dynamics.v5HM_Initial_Conditions import v5HM_initial_conditions as IC
    from scipy.integrate import solve_ivp
    event = lambda t,y,m1,m2,chi1,chi2 : y[0] - 6.0
    event.terminal = True

    M = 33.
    q = 23./10.
    S1 = 0.5
    S2 = 0.9
    f = 20
    m1 = q/(1 + q)
    m2 = 1/(1+q)
    event = lambda t,y,a,b,c,d : y[0] - 6.
    event.terminal = True
    yinit = IC(M,q,S1,S2,f)
    sol_bob = solve_ivp(rhs_bob,[0.,20000.],yinit,method = 'RK45',events = [event], args = (m1,m2,S1,S2), rtol = 1e-11, atol = 1e-12)
    sol_true = solve_ivp(rhs_true,[0.,20000.],yinit,method = 'RK45',events = [event], args = (S1,S2,m1,m2), rtol = 1e-11, atol = 1e-12)

In [10]:
if validation:
    import matplotlib.pyplot as plt
    import numpy as np
    t_bob = sol_bob.t
    t_true = sol_true.t
    r_bob = sol_bob.y[0]
    r_true = sol_true.y[0]
    plt.scatter(t_bob,r_bob,s = 1,label = 'ours',color = 'black')
    plt.plot(t_true,r_true,label = 'pyseobnr')
    plt.xlabel(r'time($M$)')
    plt.ylabel(r'$r$($M$)')
    plt.legend()
    plt.savefig('first_trajectory_eta_1_chi1_5_chi2_9.png',dpi = 300)

# Step : Implement the pyseobnr integration routine

In these sections we will document pyseobnr's exact method for integrating the trajectory.

# Step : Declare the gsl ODE solver specifications

In this section, we document the steps needed to set up the ODE solver in gsl. For more details on gsl's `odeiv2` package see [this tutorial](https://www.gnu.org/software/gsl/doc/html/ode-initval.html)
pyseobnr uses the following specifications:

- step type : rk8pd (Dormand-Prince adaptive 8th order Runge-Kutta method)
- absolute tolerance : `1e-11`
- relative tolerance : `1e-12`

In [11]:
%%writefile $Ccodesdir/v5HM_Integrator.txt

sys = odeiv2.system(v5HM_unoptimized_RHS,None,4,[m1,m2,chi1,chi2])
T = odeiv2.step_rk8pd
s = odeiv2.pygsl_lite_odeiv2_step(T,4)
atol = 1e-11
rtol = 1e-12
c = odeiv2.pygsl_lite_odeiv2_control(atol,rtol,a_y = 1,a_dydt = 1,None)
e = odeiv2.pygsl_lite_odeiv2_evolve(4)

Overwriting Dynamics/v5HM_Integrator.txt


# Step : Compute initial conditions and step-size

In this step we will call the initial conditions routine and use the stepsize choice given in line 117 of pyseobnr's [integrate_ode.py](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/eob/dynamics/integrate_ode.py):

$$
h = \frac{2\pi}{5\Omega_0 }
$$

Where, $\Omega_0$ is the starting frequency in geometric units.

$\Omega_0$ is given by:

$$
\Omega_0 = MM_\odot\pi f
$$

Where, $M$ is the total mass in solar masses and $f$ is the starting gravitational wave frequency.

In [12]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

y_init = v5HM_unoptimized_initial_conditions(M,q,chi1,chi2,f)
Omega_0 = M*4.925491025543575903411922162094833998e-6*sp.pi*f
h = 2*sp.pi/5/Omega_0
nu = q/(1 + q)/(1 + q)
m1 = q/(1 + q)
m2 = 1/(1 + q)

Appending to Dynamics/v5HM_Integrator.txt


# Step : Declare lists to store results as well as stopping condition checks

In this step we declare lists to store the primitive variables at each point as well as variables for checking the stopping conditions.

Namely, we check for 

- whether the frequency $\Omega$ has peaked; the $\Omega$ from the previous timestep is stored for this purpose
- whether the radial momentum $p_{r_{*}}$ has peaked

In [13]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

prims = []
times = []
omega_previous = Omega_0
omega_peak = False
prstar_peak = False
times.append(0.)
prims.append(y_init)

Appending to Dynamics/v5HM_Integrator.txt


# Step : Declare current state and time variables

In this step we assign the initial conditions to a state variable `y` and initial time as `t = 0.`.

In [14]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

y = y_init
t = 0.

Appending to Dynamics/v5HM_Integrator.txt


# Step : Declare provisional final time

Pyseobnr sets a provisional end time as `2.e9` (cf line 104 of [integrate_ode.py](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/eob/dynamics/integrate_ode.py).

In [15]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

t1 = 2.e9

Appending to Dynamics/v5HM_Integrator.txt


# Step : Compute the stopping radius

The stopping radius in pyseobnr is defined in the following way, given by lines 316-320 of [SEOBNRv5HM.py
](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/models/SEOBNRv5HM.py):

$$
r_{\rm stop} =
\bigg\lbrace
    \begin{array}{lr}
        0.98r_{\rm ISCO} & \Delta t_{\rm NR} > 0,\\
        -1 & \rm otherwise,
    \end{array}
$$

where, $r_{\rm ISCO}$ is the radius of the innermost stable circular orbit of the final merged Kerr black hole. $\Delta t_{\rm NR}$ is the difference in time between when the perturbing mass reaches $r_{\rm ISCO}$ and when the waveform reaches its peak amplitude. 

We first define $\Delta t_{\rm NR}$

# Step : $\Delta t_{\rm NR}$

$\Delta t_{\rm NR}$ is given in Eqs 87-90 of [SEOBNRv5HM](https://dcc.ligo.org/public/0186/T2300060/002/SEOBNRv5HM.pdf):

$$
\Delta_t = \Delta t^{\rm ns} + \Delta t^{\rm s}
\Delta t^{\rm ns} = \nu^{10.051322\nu - 1/5}\left( 55565.2392\nu^3 - 9793.17619\nu^2 - 1056.87385\nu - 59.62318  \right) \\
\Delta t^{\rm s} = \nu^{-1/5}\left( −6.789139a^{4}_{+} + 5.399623a^{3}_{+} + 6.389756a^{2}_{+}a_{−} − 132.224951a^{2}_{+}\nu + 49.801644a^{2}_{+} + 8.392389a_{+}a^{2}_{−} +179.569825a_{+}a_{−}\nu −40.606365a_{+}a_{−} +384.201019a_{+}\nu^{2} −141.253182a_{+}\nu + 17.571013a_{+} −16.905686a^{2}_{−}\nu+7.234106a^{2}_{−} +144.253396a_{−}\nu^{2} −90.192914a_{−}\nu+14.22031a_{−} \right)\\
a_{\pm} = \frac{m_1}{M}\chi_1 \pm \frac{m_2}{M}\chi_2 
$$

**Note**: There seems to be some discrepancy between the literature and pyseobnr definitions of $\Delta t_{\rm NR}$. Particularly, line 444 (aligned spin) and line 1040 (precessing spin) of [SEOBNRv5HM.py](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/models/SEOBNRv5HM.py) use:
``t_attach = t_ISCO - self.NR_deltaT`` 
whereas, Equation 41 of [SEOBNRv5HM](https://dcc.ligo.org/public/0186/T2300060/002/SEOBNRv5HM.pdf), and Equation 42 of the [Main SEOBNRv5 paper](https://journals.aps.org/prd/pdf/10.1103/PhysRevD.108.124035) indicate:

$$
t^{22}_{\rm peak} = t_{\rm ISCO} + \Delta t^{22}_{\rm ISCO}
$$

There is no discrepancy in sign between the values quoted in literature and in the code so the discrepancy is only in the literature.

Also note, when we use precession, we must choose values for $a_{\pm}$ as ${\bf l}_{N}\cdot{\bf a}_{\pm}$ evaluated when the perturber is at $r = 10M$. Why? Elaboration to follow once we begin work on precession.

In [16]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

ap = chi1*m1 + chi2*m2
am = chi1*m1 - chi2*m2
Deltatns = nu ** (-1. / 5. + 10.0513217 * nu) * (-59.62318 + -1056.87385 * nu - 9793.17619 * nu ** 2 + 55565.2392 * nu ** 3)
Deltats = nu ** (-1. / 5.) * ( 8.39238879807543 * am ** 2 * ap - 16.9056858928167 * am ** 2 * nu + 7.23410583477034 * am ** 2 + 6.38975598319936 * am * ap ** 2 + 179.569824846781 * am * ap * nu - 40.6063653476775 * am * ap + 144.253395844761 * am * nu ** 2 - 90.1929138487509 * am * nu + 14.2203101910927 * am - 6.78913884987037 * ap ** 4 + 5.39962303470497 * ap ** 3 - 132.224950777226 * ap ** 2 * nu + 49.8016443361381 * ap ** 2 + 384.201018794943 * ap * nu ** 2 - 141.253181790353 * ap * nu + 17.5710132409988 * ap)
Deltat = Deltatns + Deltats

Appending to Dynamics/v5HM_Integrator.txt


# Step : $r_{\rm ISCO}^{\rm eff}$

To define the Kerr ISCO, we also require the spin parameter $a_{f}$ of the final Kerr black hole. In this case, SEOBNRv5 uses the numerical estimate provided by Equations 12-13 of [Hoffman, et. al.](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf), hereby HBR2016. to define this, we first require an effective ISCO radius given by Equation 4-6 of HBR2016:
$$
r_{\rm ISCO}^{\rm eff} = 3 + Z_2 - \frac{a^{\rm eff}}{|a^{\rm eff}|}\sqrt{(3 - Z_1)(3 + Z_1 + 2Z_2)}\\
Z_2 = \sqrt{3 a^{\textrm{eff},2} + Z_1^2}\\
Z_1 = 1 + (1 - a^{\text{rm eff},2})^{1/3}\left[ (1 + a^{\rm eff})^{1/3} + (1 - a^{\rm eff})^{1/3} \right]\\
a^{\rm eff} = a_{\rm tot} + \xi\nu(\chi_1 + \chi_2)\\
a_{\rm tot} = m_1^2\chi_1 + m_2^2\chi_2
\xi = 0.474046
$$

Where, the value of $\xi$ is taken from Table 1 of HBR2016 with the choice of $n_M = 3, n_J = 4$ as evident in line 130 of [postadiabatic_C.pyx](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/eob/dynamics/postadiabatic_C.pyx) of the pyseobnr code.

In [17]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
atot = m1*m1*chi1 + m2*m2*chi2
aeff = atot + .474046*nu*(chi1 + chi2)
Z1eff = 1 + (np.cbrt(1 - aeff*aeff))*(np.cbrt(1 + aeff) + np.cbrt(1 - aeff))
Z2eff = np.sqrt(3*aeff*aeff + Z1eff*Z1eff)
rISCOeff = 3 + Z2eff - np.sign(aeff)*sqrt((3 - Z1eff)*(3 + Z1eff + Z2eff))

Appending to Dynamics/v5HM_Integrator.txt


# Step : $L_{\rm ISCO}^{\rm eff}$

Following Equations 2-3 of [HBR2016](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf), we define the effective angular momentum:
$$
L_{\rm ISCO}^{\rm eff} = \frac{2}{3\sqrt{3}}\left[ 1 + 2\sqrt{3r_{\rm ISCO}^{\rm eff} - 2} \right]
$$

In [18]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
LISCOeff = (2/3/np.sqrt(3))*(1 + 2*np.sqrt(3*rISCOeff - 1))

Appending to Dynamics/v5HM_Integrator.txt


# Step : $E_{\rm ISCO}^{\rm eff}$

Following Equations 2-3 of [HBR2016](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf), we define the effective binding energy:
$$
E_{\rm ISCO}^{\rm eff} = \sqrt{1 - \frac{2}{3r_{\rm ISCO}^{\rm eff}}}
$$

In [19]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
EISCOeff = np.sqrt(1 - 2/(3*rISCOeff))

Appending to Dynamics/v5HM_Integrator.txt


# Step : $l$

Following Equations 13 of [HBR2016](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf), we define the angular momentum contribution as:
$$
l = L_{\rm ISCO}^{\rm eff} - 2a_{\rm tot}(E_{\rm ISCO}^{\rm eff} - 1) + \sum_{i = 0}^{3}\sum_{j = 0}^{4} k_{ij}\nu^{2 + i}a^{\textrm{eff},j}
$$

Where, the NR coefficients $k_{ij}$ are given in Table 1 of HBR2016 for the choice of $n_M = 3, n_J = 4$.

$$
k_{00} = -5.97723\\
k_{01} = 3.39221\\
k_{02} = 4.48865\\
k_{03} = -5.77101\\
k_{04} = -13.0459\\
k_{10} = 35.1278\\
k_{11} = -72.9336\\
k_{12} = -86.0036\\
k_{13} = 93.7371\\
k_{14} = 200.975\\
k_{20} = -146.822\\
k_{21} = 387.184\\
k_{22} = 447.009\\
k_{23} = -467.383\\
k_{24} = -884.339\\
k_{30} = 223.911\\
k_{31} = -648.502\\
k_{32} = -697.177\\
k_{33} = 753.738\\
k_{34} = 1166.89
$$

In [20]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
k = np.zeros([4,5])
k[0,0] = -5.97723
k[0,1] = 3.39221
k[0,2] = 4.48865
k[0,3] = -5.77101
k[0,4] = -13.0459
k[1,0] = 35.1278
k[1,1] = -72.9336
k[1,2] = -86.0036
k[1,3] = 93.7371
k[1,4] = 200.975
k[2,0] = -146.822
k[2,1] = 387.184
k[2,2] = 447.009
k[2,3] = -467.383
k[2,4] = -884.339
k[3,0] = 223.911
k[3,1] = -648.502
k[3,2] = -697.177
k[3,3] = 753.738
k[3,4] = 1166.89

NRfactor = 0
nu_2plusi = nu*nu
for i in range(len(k)):
    atot_j = 1 
    for j in range(len(k[i])):
        NRfactor += k[i,j]*nu_2pi*aeff_j
        atot_j *= atot
    nu_2plusi *= nu
ell = np.abs(LISCOeff - 2*atot*(EISCOeff - 1) + NRfactor)

Appending to Dynamics/v5HM_Integrator.txt


# Step : $a_{f}$

Following Equations 13 of [HBR2016](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf), we define the final spin:
$$
a_{f} = a_{\rm tot} + \nu l
$$

In [21]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
af = atot + nu*l

Appending to Dynamics/v5HM_Integrator.txt


# Step : $r_{\rm ISCO}$

Equations 4-6 of [HBR2016](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf) are a general formula for the radius of the ISCO. In this case, we use the final spin calculated above:
$$
r_{\rm ISCO} = 3 + Z_2 - \frac{a_{f}}{|a_{f}|}\sqrt{(3 - Z_1)(3 + Z_1 + 2Z_2)}\\
Z_2 = \sqrt{3 a_{f}^{2} + Z_1^2}\\
Z_1 = 1 + (1 - a_{f}^{2})^{1/3}\left[ (1 + a_{f})^{1/3} + (1 - a_{f})^{1/3} \right]\\
$$

In [22]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
Z1 = 1 + (np.cbrt(1 - af*af))*(np.cbrt(1 + af) + np.cbrt(1 - af))
Z2 = np.sqrt(3*af*af + Z1*Z1)
rISCO = 3 + Z2 - np.sign(af)*sqrt((3 - Z1f)*(3 + Z1f + Z2f))

Appending to Dynamics/v5HM_Integrator.txt


# Step : $r_{\rm stop}$

As defined earlier, the stopping radius is given by:

$$
r_{\rm stop} =
\bigg\lbrace
    \begin{array}{lr}
        0.98r_{\rm ISCO} & \Delta t_{\rm NR} > 0,\\
        -1 & \rm otherwise,
    \end{array}
$$


In [23]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
rstop = -1
if Deltat > 0:
    rstop = 0.98*rISCO

Appending to Dynamics/v5HM_Integrator.txt


# Step : $M_{f}$

In accordance with pyseobnr, cf line 178 of [compute_hlms.py](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/eob/waveform/compute_hlms.py), [Jimenez-Forteza et. al](https://arxiv.org/pdf/1611.00332.pdf), hereby UIB2016, is used to compute the mass of the final black hole. From the discussion above Equation 4 and Equation 28 of UIB2016 we find:

$$
M_f = 1 - E_{\rm rad}\\
E_{\rm rad} \equiv E_{\rm rad}(\nu,\hat{S},\Delta\chi) = E_{\rm rad}(\nu,\hat{S}) + \Delta E_{\rm rad}(\eta,\hat{S},\Delta\chi) 
$$
Where, $E_{\rm rad}$ is the energy radiated through gravitational waves.
We derive all the quantities and build the final mass.

# Step : $\hat{S}$

We define $\hat{S}$ as per Equation 1 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf) as:

$$
\hat{S} = \frac{m_1^2\chi_1 + m_2^2\chi_2}{m_1^2 + m_2^2}
$$

In [24]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
Shat = (m1*m1*chi1 + m2*m2*chi2)/(m1*m1 + m2*m2)
Shat2 = Shat*Shat
Shat3 = Shat2*Shat
Shat4 = Shat3*Shat
nu2 = nu*nu
nu3 = nu2*nu
nu4 = nu3*nu
sqrt1m4nu = np.sqrt(1. - 4.*nu)

Appending to Dynamics/v5HM_Integrator.txt


# Step : $\Delta \chi$

We define $\Delta \chi$ as per the discussion below Equation 1 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf):

$$
\Delta\chi = \chi_1-\chi_2
$$

In [25]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
Deltachi = chi1-chi2
Deltachi2 = Deltachi*Deltachi

Appending to Dynamics/v5HM_Integrator.txt


# Step : $E_{\rm rad}(\nu,0)$

We define the non-spinning fit to the radiated energy as per Equation 21 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf):

$$
E_{\rm rad}(\nu,\hat{S} = 0) = a_4\nu^4 + a_3\nu^3 + a_2\nu^2 + \left(1 - \frac{2\sqrt{2}}{3}\right)\nu 
$$

Where, the coefficients are given in this [arXiv ancilliary file](https://arxiv.org/src/1611.00332v2/anc/FinalStateUIB2016_suppl_Erad_coeffs.txt):

$$
a_2 = 0.5609904135313374\\
a_3 = -0.84667563764404 \\
a_4 = 3.145145224278187 
$$

In [26]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
a2 = 0.5609904135313374 
a3 = -0.84667563764404
a4 = 3.145145224278187
Erad_nu_0 = a4*nu4 + a3*nu3 + a2*nu2 + (1. - 2.*np.sqrt(2.)/3.)*nu

Appending to Dynamics/v5HM_Integrator.txt


# Step : $E_{\rm rad}(0.25,\hat{S})$ coefficients

Before implementing the spin contributions to the radiated energy, we should implement the modified coefficients. That is, the non-spinning and spinning contributions represent one dimensional fits to numerical relativity data at the higher end of the mass ratio. In order to develop a two dimensional fit in mass ratio and spin as well as to include effects at extreme mass ratio, coefficients for the spinning 1-D fit are modified (cf discussion above Equation 9 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf)).

The coefficients are defined in this [arXiv ancilliary file](https://arxiv.org/src/1611.00332v2/anc/FinalStateUIB2016_suppl_Erad_coeffs.txt).


$$
b_1 = -0.2091189048177395 \\
b_2 = -0.19709136361080587 \\
b_3 = -0.1588185739358418 \\
b_5 = 2.9852925538232014 \\
f_{20} = 4.271313308472851 \\
f_{30} = 31.08987570280556 \\
f_{50} = 1.5673498395263061 \\
f_{10} = 1.8083565298668276 \\
f_{21} = 0. \\
f_{11} = 15.738082204419655 \\
f_{31} = -243.6299258830685 \\
f_{51} = -0.5808669012986468 \\
f_{j2} = 16 - 16f_{j0} - 4f_{j1} \\
$$

The final coefficients given by Equation 9 of UIB2016:

$$
b^{\rm fin}_{i} = b_{i}\sum{j = 0}^{2} f_{ij}\nu^j
$$

In [27]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
b1 = -0.2091189048177395
b2 = -0.19709136361080587
b3 = -0.1588185739358418
b5 = 2.9852925538232014
f20 = 4.271313308472851
f30 = 31.08987570280556
f50 = 1.5673498395263061
f10 = 1.8083565298668276
f21 = 0.
d10 = -0.09803730445895877
d11 = -3.2283713377939134
d20 = 0.01118530335431078
d30 = -0.01978238971523653
d31 = -4.91667749015812
f11 = 15.738082204419655
f31 = -243.6299258830685
f51 = -0.5808669012986468
bfin1 = b1*(f10 + f11*nu + (16. - 16.*f10 - 4.*f11)*nu2)
bfin2 = b2*(f20 + f21*nu + (16. - 16.*f20 - 4.*f21)*nu2)
bfin3 = b3*(f30 + f31*nu + (16. - 16.*f30 - 4.*f31)*nu2)
bfin5 = b5*(f50 + f51*nu + (16. - 16.*f50 - 4.*f51)*nu2)

Appending to Dynamics/v5HM_Integrator.txt


# Step : $E_{\rm rad}(0.25,\hat{S})$

We define the spin contributions fit to the radiated energy as per Equation 22 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf):

$$
E_{\rm rad}(\nu = 0.25,\hat{S}) = \frac{1 + 0.346b_1^{\rm fin}\hat{S} + 0.211b_2^{\rm fin}\hat{S}^2 + 0.128b_3^{\rm fin}\hat{S}^3}{1 - 0.212b_5^{\rm fin}\hat{S}} 
$$

Note: We do not use the multiplicative factor of 0.0484161 as it will be normalized away in the final fit (cf Equation 23 of UIB2016)

In [28]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
Erad_eq_Shat = (0.128*bfin3*Shat3 + 0.211*bfin2*Shat2 + 0.346*bfin1*Shat + 1.)/(1 - 0.212*bfin5*Shat)

Appending to Dynamics/v5HM_Integrator.txt


# Step : $E_{\rm rad}(\nu,\hat{S})$

We define the 2-D fit to the radiated energy as per Equation 23 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf):

$$
E_{\rm rad}(\nu,\hat{S}) = E_{\rm rad}(\nu,0)E_{\rm rad}(0.25,\hat{S})
$$

In [29]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
Erad_nu_Shat = Erad_nu_0*Erad_eq_Shat

Appending to Dynamics/v5HM_Integrator.txt


# Step : $\Delta E_{\rm rad}(\nu,\hat{S},\Delta\chi)$

We define the 3-D correction to the radiated energy as per Equation 27 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf) and the discussion above.:

$$
\Delta E_{\rm rad}(\nu,\hat{S},\Delta \chi) = A_1\Delta \chi + A_2 \Delta\chi^2 + A_3\hat{S}\Delta\chi\\
A_1 = d_{10}\sqrt{1 - 4\nu}\nu^2(d_{11}\nu + 1) \\
A_2 = d_{20}\nu^3 \\
A_3 = d_{30}\sqrt{1 - 4\nu}\nu(d_{31}\nu + 1) \\
$$

The coefficients are given in this [arXiv ancilliary file](https://arxiv.org/src/1611.00332v2/anc/FinalStateUIB2016_suppl_Erad_coeffs.txt):

$$
d_{10} = -0.09803730445895877 \\
d_{11} = -3.2283713377939134 \\
d_{20} = 0.01118530335431078 \\
d_{30} = -0.01978238971523653 \\
d_{31} = -4.91667749015812 \\
$$

In [30]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
d10 = -0.09803730445895877
d11 = -3.2283713377939134
d20 = 0.01118530335431078
d30 = -0.01978238971523653
d31 = -4.91667749015812
A_1 = d10*sqrt1m4nu*nu2*(d11*nu+1)
A_2 = d20*nu3
A_3 = d30*sqrt1m4nu*nu*(d31*nu + 1)
DeltaErad_nu_Shat_Deltachi = A1*Deltachi + A2*Deltachi2 + A3*Deltachi*shat

Appending to Dynamics/v5HM_Integrator.txt


# Step : $M_f$

We define the full fit to the radiated energy as per Equation 28 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf) and thus, the final mass as per Equation 4 of the same:

$$
E_{\rm rad}(\nu,\hat{S},\Delta\chi) = E_{\rm rad}(\nu,\hat{S}) + \Delta E_{\rm rad}(\nu,\hat{S},\Delta\chi)\\
M_f = 1 - E_{\rm rad}(\nu,\hat{S},\Delta \chi)
$$

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt
M_f = 1 - (Erad_nu_Shat + DeltaErad_nu_Shat_Deltachi)

# Step : Begin the integration loop and take a step

In this step, we begin the integration loop and perform the first step, appending the results.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

while t < t1:
    status, t, h, y = e.apply(c,s,sys,t,t1,h,y)
    if status != errno.GSL_SUCCESS:
            print("break status", status)
            break
    prims.append(y)
    times.append(t)
    

# Step : Do a stopping condition check

In this step, we check for the stopping conditions. First, we check is the radius is less than 6M.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

    if r < = 6:

# Step : Compute the right hand sides

In this step, we compute the right hand sides in order to do the stopping condition checks.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

        rhs = v5HM_unoptimized_RHS(t,y,m1,m2,chi1,chi2)
        drdt = rhs[0]
        dphidt = rhs[1]
        dprstardt = rhs[2]

# Step : Check for a frequency peak

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

        if dphidt < omega_previous:
            omega_peak = True
            break

# Step : Check for a positive radial derivative

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

        if drdt > 0:
            break

# Step : Check for a positive radial momentum derivative

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

        if dprdt > 0:
            prstar_peak = True
            break

# Step : 